In [1]:
import os
import sys

import numpy as np
import pandas as pd
import tensorflow as tf

# BERT
bert_path = "C:/Users/roksi/Google Drive/NLU/Projects/project_2/venv/Lib/site-packages/bert"

if not bert_path in sys.path:
    sys.path.append(bert_path)

import modeling
import tokenization


data_path = "./data/"

data_val = pd.read_csv(data_path + "cloze_test_val__spring2016 - cloze_test_ALL_val.csv")
data_train = pd.read_csv(data_path + "train_stories.csv")

In [2]:
def pp_text_a(row):
    return row[1] + " " + row[2] + " " + row[3] + " " + row[4]
    
def pp_text_b_pos(row):
    if row[7] == 1:
        return row[5]
    elif row[7] == 2:
        return row[6]
    
def pp_text_b_neg(row):
    if row[7] == 1:
        return row[6]
    elif row[7] == 2:
        return row[5]

In [4]:
df_pos = pd.DataFrame()
df_pos["unique_id"] = data_val["InputStoryid"]
df_pos["text_a"] = data_val.apply(pp_text_a, axis=1)
df_pos["text_b"] = data_val.apply(pp_text_b_pos, axis=1)
df_pos["label"] = "right" # 1 -> positive/correct class

df_neg = pd.DataFrame()
df_neg["unique_id"] = data_val["InputStoryid"]
df_neg["text_a"] = data_val.apply(pp_text_a, axis=1)
df_neg["text_b"] = data_val.apply(pp_text_b_neg, axis=1)
df_neg["label"] = "wrong"

df = pd.concat([df_pos, df_neg], axis=0).sample(frac=1).reset_index(drop=True)
print(df["text_a"][0])
print(df["text_b"][0])
print(df["label"][0])

# df.apply(lambda row: len(row[1]), axis=1).max()

Tim always wanted to play the drums. His parents bought him a set for Christmas. He would play them all the time. It was way too loud and upset his parents.
They set up a space in the garage for him to practice.
right


In [6]:
df.to_csv("./data/preprocessed/sct.train.tsv", sep="\t", header=True, index=False)
df.to_csv("./data/preprocessed/sct.validation.tsv", sep="\t", header=True, index=False)

In [108]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = bert_path + "/bert/"

MAX_SEQ_LENGTH = 128
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CHKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
BATCH_SIZE = 128


print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))

***** BERT pretrained directory: C:/Users/roksi/bert/bert/ *****


In [140]:
from run_classifier import InputExample
from run_classifier import convert_examples_to_features
from run_classifier import input_fn_builder

MAX_SEQ_LENGTH = 256
label_list = [0, 1]

tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=True)

df_InputExamples = df.apply(lambda x: InputExample(guid=x[0], text_a=x[1], text_b=x[2], label=x[3]), axis=1)

df_features = convert_examples_to_features(df_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 12
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 449c1058-fb27-4c6d-9cac-fbd5b32b80da
INFO:tensorflow:tokens: [CLS] billy buried some treasure in his backyard . he drew a map so he wouldn ' t forget where he ' d put it . years passed and billy ' s family moved . decades later , he went back and used his map to dig it back up . [SEP] he threw away the treasure he found . [SEP]
INFO:tensorflow:input_ids: 101 5006 3950 2070 8813 1999 2010 16125 1012 2002 3881 1037 4949 2061 2002 2876 1005 1056 5293 2073 2002 1005 1040 2404 2009 1012 2086 2979 1998 5006 1005 1055 2155 2333 1012 5109 2101 1010 2002 2253 2067 1998 2109 2010 4949 2000 10667 2009 2067 2039 1012 102 2002 4711 2185 1996 8813 2002 2179 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:tensorflow:label: 1 (id = 1)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: c2699d68-4f16-44d1-bff6-6a66ec266a46
INFO:tensorflow:tokens: [CLS] mike had a free saturday . he drove to several car dealers . he was interested in seeing potential future cars for himself . mike found a jeep he fell in love with . [SEP] mike never liked jeep ##s . [SEP]
INFO:tensorflow:input_ids: 101 3505 2018 1037 2489 5095 1012 2002 5225 2000 2195 2482 16743 1012 2002 2001 4699 1999 3773 4022 2925 3765 2005 2370 1012 3505 2179 1037 14007 2002 3062 1999 2293 2007 1012 102 3505 2196 4669 14007 2015 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [156]:
# from extract_features import read_examples

tf.reset_default_graph()

tf.logging.set_verbosity(tf.logging.INFO)

layer_indexes = [-1, -2]

bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=True)

# examples = read_examples("tmp_input.txt")





In [141]:
BERT_CONFIG

'C:/Users/roksi/bert/bert//bert_config.json'

In [14]:
df

,guid,text_a,text_b,label
0,138d5bfb-05cc-41e3-bf2c-fa85ebad14e2,Rick grew up in a troubled household. He never...,He is happy now.,1
1,bff9f820-9605-4875-b9af-fe6f14d04256,Laverne needs to prepare something for her fri...,The brownies are so delicious Laverne eats two...,1
2,e8f628d5-9f97-40ed-8611-fc0e774673c4,Sarah had been dreaming of visiting Europe for...,Sarah decided that she preferred her home over...,1
3,f5226bfe-9f26-4377-b05f-3d9568dbdec1,Gina was worried the cookie dough in the tube ...,Gina liked the cookies so much she ate them al...,1
4,69ac9b05-b956-402f-9fff-1f926ef9176b,It was my final performance in marching band....,I was very proud of my performance.,1
5,0f65bab6-8165-4361-980a-117046569fe2,I had been giving this homeless man change eve...,I never gave the man money again.,1
6,d80cabdd-7a85-47e3-86be-5ce6591ca51e,Jim found an old disposable camera in the bott...,Jim took time to decide what he would take a p...,1
7,58090d3f-8a91-4c89-83ef-2b4994de9d24,Ron started his new job as a landscaper today....,His boss commends him for a job well done.,1
8,e17053ac-2046-48c8-a7a2-7b9509c10e64,John and Billy became very skilled at beer pon...,"In Vegas, John and Billy competed against eigh...",1
9,69b26ae4-b778-4cd1-9f13-27d28fd4430e,Caroline was a student in medical school. Caro...,"Later, she passed the test.",1


Billy buried some treasure in his backyard. He drew a map so he wouldn't forget where he'd put it. Years passed and Billy's family moved. Decades later, he went back and used his map to dig it back up. He threw away the treasure he found.
Ellen used to live in the country. She misses having days with the power out. One day an ice storm caused the power to go out in her city home. She got very cold. Ellen was perfectly comfortable.
Ellen used to live in the country. She misses having days with the power out. One day an ice storm caused the power to go out in her city home. She got very cold. She was happy when the power came back on.
I went to buy a new television last week. After finally picking one out I took it home. I noticed it was broken as soon as I got it out of the box. The screen was cracked. I returned the TV.
Mike had a free Saturday. He drove to several car dealers. He was interested in seeing potential future cars for himself. Mike found a Jeep he fell in love with. Mike n

In [160]:
features = df_features

unique_id_to_feature = {}
for feature in features:
    unique_id_to_feature[feature.guid] = feature


AttributeError: 'InputFeatures' object has no attribute 'guid'

In [163]:
features[0].is_real_example

True

In [ ]:
from extract_features import model_fn_builder
model_fn = model_fn_builder()